### Collisional-radiative model for Argon Plasma
#### Kinetics fitting
See: https://doi.org/10.1063/1.3585688

In [1]:
import os
import copy
import numpy as np
import dill as pickle
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

Inputs

In [2]:
path_to_kin = "./../rates/kin_raw.p"
path_to_rad = "./../rates/rad_raw.p"

In [3]:
# Save path
out_path = "./../rates/"
os.makedirs(out_path, exist_ok=True)

Fit functions and parameters

In [4]:
ARR_PARAM = ("A", "beta", "Ta")

# Arrhenius law - Linear
def fun(T, *args):
  A, beta, Ta = args
  return A * np.exp(beta*np.log(T) - Ta/T)

# Arrhenius law - Log
def fun_ln(T, *args):
  logA, beta, Ta = args
  return logA + beta*np.log(T) - Ta/T

# Arrhenius law Jacobian - Log
def jac_ln(T, *args):
  return np.vstack([np.ones_like(T), np.log(T), -1/T]).T

# Curve fit arguments
cfit_args = {
  "f": fun_ln,
  "p0": [-20, 0, 1e3],
  "method": "trf",
  "jac": jac_ln,
  "bounds": (
    [-np.inf, -np.inf,   0], # Lower bounds
    [     -5,  np.inf, 1e6]  # Upper bounds
  )
}

# Fitting functions
def fit_rate(T, k):
  indices = k > 0
  T = T[indices]
  k = np.log(k[indices])
  if (len(T) < 4):
    param = np.zeros((3,))
  else:
    param = curve_fit(
      xdata=T,
      ydata=k,
      **cfit_args
    )[0]
    param[0] = np.exp(param[0])
  return param

def fit_rates(T, rates):
  shape = rates.shape[:-1]
  rates = rates.reshape(-1, len(T))
  param = np.stack([fit_rate(T, k) for k in rates], 1)
  param = {p: param[i].reshape(shape) for i, p in enumerate(ARR_PARAM)}
  return param

**Kinetics**

In [5]:
kin_raw = pickle.load(open(path_to_kin, "rb"))
T = kin_raw["T"]

In [6]:
kin_fit = copy.deepcopy(kin_raw)
for (name, rates) in kin_raw.items():
  if (name not in ("T", "EN", "EI")):
    kin_fit[name]["values"] = fit_rates(T, rates["values"])

**Radiation**

In [7]:
rad_raw = pickle.load(open(path_to_rad, "rb"))
T = rad_raw["T"]

In [8]:
rad_fit = copy.deepcopy(rad_raw)
rad_fit["PI"]["values"] = fit_rates(T, rad_raw["PI"]["values"])

Compare fit rates

In [9]:
FIT_RATES = ("PI", "EXe", "EXh", "Ie", "Ih")

In [10]:
def pick_rate():
  name = np.random.choice(FIT_RATES)
  dtb = rad_fit if (name == "PI") else kin_fit
  size = dtb[name]["values"]["A"].size
  rate = 0.0
  while rate == 0.0:
    i = np.random.choice(size)
    rate = dtb[name]["values"]["A"].reshape(-1)[i]
  return name, i

def compute_rates(name, i):
  dtb_fit = rad_fit if (name == "PI") else kin_fit
  dtb_raw = rad_raw if (name == "PI") else kin_raw
  param = [dtb_fit[name]["values"][p].reshape(-1)[i] for p in ARR_PARAM]
  k_fit = fun(T, *param)
  k_raw = dtb_raw[name]["values"].reshape(-1, len(T))[i]
  return k_fit, k_raw

In [ ]:
for i in range(20):
  name, i = pick_rate()
  k_fit, k_raw = compute_rates(name, i)
  plt.plot(T, k_fit, label="Fit")
  plt.plot(T, k_raw, label="Raw")
  plt.title(f"{name}, i = {i} - Linear")
  plt.legend()
  plt.show()
  plt.close()
  plt.semilogy(T, k_fit, label="Fit")
  plt.semilogy(T, k_raw, label="Raw")
  plt.title(f"{name}, i = {i} - Log")
  plt.legend()
  plt.show()
  plt.close()

Save fit rates

In [12]:
pickle.dump(kin_fit, open(out_path + "/kin_fit.p", "wb"))
pickle.dump(rad_fit, open(out_path + "/rad_fit.p", "wb"))